# Il problema delle $n$ regine

Nel gioco degli scacchi, la regina può muoversi in orizzontale, verticale e diagonale e di un numero qualunque di celle. Posizionare il massimo numero regine su una scacchiera $n\times n$ in modo che nessuna possa essere mangiata in una sola mossa. <br />
<b>Mangiare:</b> se la regina si sposta in diagonale, mangia tutti quelli sulla diagonale, in orizz. e in vert.

# Soluzione

![Immagine (da Wikipedia)](https://upload.wikimedia.org/wikipedia/commons/2/2c/Eight_Queens12_positions.gif)

A differenza del Sudoku, questo problema è di ottimizzazione, ovvero dobbiamo trovare il numero massimo di regine. È da notare che ci può essere al più una regina per riga, quindi $n$ è un _upper bound_, ovvero una stima per eccesso, della soluzione ottima.

Il problema ha una sola decisione di tipo sì/no in ogni cella, quindi ci servono $n^2$ variabili binarie.

In [1]:
!pip install mip  # Solo se si usa colab.research.google.com


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [22]:
import mip

m = mip.Model()

In [24]:
# Variabili: una per ogni cella, quindi due indici i e j ciascuno da 0 a n-1
# Facciamo una lista di liste
n=10
N=range(n)
x=[[m.add_var(var_type=mip.BINARY) for j in N] for i in N]
# x[i][j]


In [25]:
# Funzione obiettivo: numero di regine
m.objective=mip.maximize(mip.xsum(x[i][j] for i in N for j in N))
m.optimize()

Cgl0004I processed model has 20 rows, 100 columns (100 integer (100 of which binary)) and 200 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 4.975%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I MIPStart provided solution with cost 10
Cbc0012I Integer solution of 10 found by Reduced search after 0 iterations and 0 nodes (0.00 seconds)
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of -10 already found by heuristic
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0000I Optimal - objective value 100
Clp0032I Optimal objective 100 - 0 iterations time 0.002, Idiot 0.00

Starting MIP optimization


<OptimizationStatus.OPTIMAL: 0>

In [11]:
# Vincolo 1: per ogni riga c'è al più una regina
for i in N:
    m.add_constr(mip.xsum(x[i][j] for j in N)<=1)

In [12]:
# Vincolo 2: per ogni colonna c'è al massimo una regina
for j in N:
    m.add_constr(mip.xsum(x[i][j] for i in N)<=1)

In [28]:
# Vincolo 3a e 3b: su ciascuna diagonale (verso nord-est e verso nord-ovest) c'è al massimo una regina
for k in range(2,2*n-2):
    m.add_constr(mip.xsum(x[i][j] for i in N for j in N if i+j==k)<=1)
for k in range(-n+1,n):
    m.add_constr(mip.xsum(x[i][j] for i in N for j in N if i-j==k)<=1)

m.optimize()
print(f'{m.objective_value} regine')
for i in N:
    for j in N:
        if (x[i][j].x > 0.5):
            print('*',end='')
        else:
            print('.',end='')
        print()
#for i in N:
#    for j in N:
#        if(x[i][j]==1):
#            for h in range(0,n-i):
#                x[i+h][j+h]=0;

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 33 (-2) rows, 98 (-102) columns and 192 (-4) elements
Clp1000I sum of infeasibilities 1.6376e-07 - average 4.96242e-09, 15 fixed columns
Coin0506I Presolve 33 (0) rows, 83 (-15) columns and 166 (-26) elements
Clp0006I 0  Obj 14.682362 Dual inf 8299.9999 (83)
Clp0029I End of values pass after 83 iterations
Clp0014I Perturbing problem by 0.001% of 1.0000009 - largest nonzero change 2.556097e-05 ( 0.0012780485%) - largest zero change 2.8691903e-05
Clp0000I Optimal - objective value 16
Clp0000I Optimal - objective value 16
Coin0511I After Postsolve, objective 16, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 16 Dual 